#  DeploymenT and inference

In [3]:
import os
import sys

current_dir = os.getcwd()
kit_dir =  os.path.abspath(os.path.join(current_dir, '..'))
repo_dir = os.path.abspath(os.path.join(kit_dir, '..'))
sys.path.append(repo_dir)

from utils.fine_tuning.src.snsdk_wrapper import SnsdkWrapper

## Step by Step / Manual setting

In [4]:
sambastudio_client = SnsdkWrapper()

2024-11-25 11:41:55,648 [INFO] Using variables from Snapi config to set up Snsdk.


### Create Project

In [ ]:
project = {
    'project_name': 'byoc fine-tuning project',
    'project_description': 'this project will be used to test the BYOC and Fine-tuning e2e pipeline implementation'
}

In [ ]:
sambastudio_client.create_project(
    project_name = project['project_name'],
    project_description = project['project_description']
)

2024-11-22 18:57:48,601 [INFO] Project with name 'byoc fine-tuning project' found with id 08b2e9e9-cebe-4f57-9271-a7e6c6f1561d
2024-11-22 18:57:48,601 [INFO] Project with name 'byoc fine-tuning project' already exists with id '08b2e9e9-cebe-4f57-9271-a7e6c6f1561d', using it


'08b2e9e9-cebe-4f57-9271-a7e6c6f1561d'

### Create a Bundle model

#### Set bundle model config 

In [ ]:
composite_model= {
    'model_name':  'Suzume-Llama-3-8B-Multilingual-Bundle',
    'description': 'Bundle including base and public health fine-tuned Suzume-Llama-3-8B-Multilingual',
    'rdu_required': 8,
    'model_version': '1',
    'model_list':
        - 'Suzume-Llama-3-8B-Multilingual'
        - 'Suzume-Llama-3-8B-Multilingual-Publichealth'
}

In [ ]:
sambastudio_client.create_composite_model(       
        model_name = composite_model['model_name'],
        description = composite_model['description'],
        model_list = composite_model['model_list'],
        rdu_required = composite_model['rdu_required']
)

### Create Endpoint

#### Set endpoint config 

In [ ]:
endpoint = {
  'endpoint_name': 'suzume-multilingual-endpoint',
  'endpoint_description': 'endpoint for suzume multilingual bundle',
  'endpoint_instances': 1,
  'hyperparams': {}
}

In [ ]:
sambastudio_client.create_endpoint(
    project_name=project['project_name'],
    endpoint_name=endpoint['endpoint_name'],
    endpoint_description=endpoint['endpoint_description'],
    model_name=composite_model['model_name'],
    model_version=composite_model['model_version'],
    instances=endpoint['endpoint_instances'],
    hyperparams=endpoint['hyperparams'],
    rdu_arch="SN40L-8",
)

In [ ]:
sambastudio_client.get_endpoint_details(
    project_name="example project",
    endpoint_name="test-endpoint-sql"
)

### Inference

#### Get endpoint details

In [ ]:
endpoint_env = sambastudio_client.get_endpoint_details(
    project_name=project['project_name'],
    endpoint_name=endpoint['endpoint_name']["langchain_wrapper_env"]
)

In [ ]:
from langchain_community.chat_models import ChatSambaStudio

multilingual_llm = ChatSambaStudio(
    sambastudio_url=endpoint_env.get("SAMBASTUDIO_URL"),
    sambastudio_api_key=endpoint_env.get("SAMBASTUDIO_API_KEY"),
    temperature = 0.01,
    max_tokens = 1024,
    process_prompt = False,
    model = "Suzume-Llama-3-8B-Multilingual"
)

multilingual_health_llm = ChatSambaStudio(
    sambastudio_url=endpoint_env.get("SAMBASTUDIO_URL"),
    sambastudio_api_key=endpoint_env.get("SAMBASTUDIO_API_KEY"),
    temperature = 0.01,
    max_tokens = 1024,
    process_prompt = False,
    model = "Suzume-Llama-3-8B-Multilingual-Publichealth"
)

In [ ]:
multilingual_llm.invoke("which are the best habits to improve my mental capabilities")

In [ ]:
multilingual_health_llm.invoke("Why im felling mental fatigue, can be covid related")

## Streamlined Execution

In [ ]:
config_file = os.path.join(repo_dir, 'finetune_config.yaml')
sambastudio_client = SnsdkWrapper(config_file)
sambastudio_client.create_project()
sambastudio_client.create_composite_model()
sambastudio_client.create_endpoint()
endpoint_env = sambastudio_client.get_endpoint_details(
    project_name=project['project_name'],
    endpoint_name=endpoint['endpoint_name']["langchain_wrapper_env"]
)